In [32]:
import requests
import yaml

def get_yaml_content(file_url):
    try:
        # Send a GET request to the URL
        response = requests.get(file_url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Parse the response content as YAML
            return yaml.safe_load(response.content)
        else:
            print(f"Failed to retrieve file: Status code {response.status_code}")
            return None
    except requests.exceptions.RequestException as e:
        # Handle any exceptions that occur during the request
        print(f"An error occurred: {e}")
        return None
    except yaml.YAMLError as e:
        # Handle any exceptions that occur during YAML parsing
        print(f"Failed to parse YAML: {e}")
        return None

In [33]:
url ='https://raw.githubusercontent.com/marctheshark3/ergo-fan-clubs/main/pool/conf.yaml'

In [34]:
get_yaml_content(url)

{'background_color': 'orange',
 'secondary_color': 'black',
 'large_text_color': 'white',
 'small_text_color': 'white',
 'shares': 'min_payout.png',
 'balance': 'triangle.png',
 'total_paid': 'ergo.png',
 'last_payment': 'coins.png',
 'price': 'ergo.png',
 'schema': 'ergo.png',
 'participation': 'percentage.png',
 'paid_today': 'ergo.png',
 'payment_link': 'ergo.png',
 'minimum_payout': 'min_payout.png',
 'pool_fee': 'percentage.png',
 'pool_hashrate': 'boltz.png',
 'pool_effort': 'ergo.png',
 'num_blocks_found': 'ergo.png',
 'miners_online': 'smileys.png',
 'network_hashrate': 'bolt.png',
 'network_difficulty': 'gauge.png',
 'block_height': 'height.png',
 'ergo': 'ergo.png',
 'sigusd': 'ergo.png',
 'rosen': 'rosen.png',
 'comet': 'ergo.png',
 'cyberverse': 'ergo.png',
 'rsBTC': 'ergo.png',
 'rsADA': 'ergo.png',
 'bobber': 'ergo.png',
 'love': 'ergo.png',
 'block_found': 'block_found.gif'}

In [14]:
def set_theme(url):
    data = get_yaml_content(url)

    card_color = config['secondary_color'] # '#27374D'
    background_color = config['background_color'] #'#526D82'
    large_text_color = config['large_text_color'] # '#9DB2BF' 
    small_text_color = config['small_text_color'] # '#DDE6ED'

    top_card_style = {'backgroundColor': card_color, 'color': small_text_color, 'padding': '20px',}
    
    card_style = {'border': '1px solid {}'.format('#292929'), 'backgroundColor': card_color, 'color': 'white',
                  'padding': '15x', 'textAlign': 'center', 'justifyContent': 'center'}
    
    table_style = {'backgroundColor': card_color, 'color': large_text_color,
                   'fontWeight': 'bold', 'textAlign': 'center', 'border': '1px solid black',}
    
    top_row_style = {'backgroundColor': card_color,
                     'color': 'white',
                     'display': 'flex',
                     'padding': '20px',
                     'height': 'auto',
                     'justifyContent': 'flex-start',}
    
    bottom_row_style = {
                        'backgroundColor': card_color,
                        'color': 'white',
                        'padding': '20x',
                        'display': 'flex',
                        'alignItems': 'left',
                        'justifyContent': 'left',
                        'fontSize': '14px',
                    }
    return top_card_style, card_style, table_style, top_row_style, bottom_row_style
    
    
def create_row_card(h2_text, p_text, style, image=None):
    return dbc.Col(dbc.Card(style, children=[
        # html.Img(src=image, style=top_image_style),
        html.H2(h2_text, style={'color': '#ff5e18'}),
        html.P(p_text)]),)
    
def create_image_text_block(text, image=None, block_style=bottom_row_style, image_style=bottom_image_style):
    if image:
        return html.Div(style=block_style, children=[
                        html.Img(src='assets/{}'.format(image), style=image_style),
                        html.Span(text, style={'padding': '10px', 'width': '100%', 'height': 'auto'})])
    else:
        return html.Div(style=block_style, children=[
                        html.Span(text, style={'padding': '10px', 'width': '100%', 'height': 'auto'})])

NameError: name 'bottom_row_style' is not defined

In [ ]:
52